In [62]:
import os
import numpy as np
from PIL import Image
import cv2
from utils import preview
from imgaug import augmenters as iaa
import datetime
import json

In [2]:
def load_images(image_dir):
    """
    Load transparent object images from a directory
    """
    # Image file paths
    image_files = os.listdir(image_dir)
    image_files.remove('.DS_Store')
    image_files = [os.path.join(image_dir, f) for f in image_files] 

    # Load images
    images = []
    for f in image_files:
        img = cv2.imread(f, cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
        images.append(img)

    return images

In [3]:
def pad_image(img, padding=10, constant=0):
    """
    Pad a 2D image with a constant value
    Arguments:
    - img - numpy array - image to pad
    - padding - int - padding to apply
    - constant - int - constant value to pad with
    """
    img = img.copy()
    img_padded = np.ones(
        (img.shape[0] + padding*2, img.shape[1] + padding*2, img.shape[2]),
        dtype=img.dtype
    ) * constant
    img_padded[padding:-padding, padding:-padding] = img
    return img_padded

In [4]:
def restore_alpha(image, crop=False):
    """
    Restores the alpha channel to 0 or 255 after image augmentation distorts it
    - crop - bool - whether to crop the image using the alpha channel
    """
    image = image.copy()
    
    # Binarise the alpha channel
    alpha_channel = image[:, :, 3]
    alpha_channel = (alpha_channel >= 128) * 255
    image[:, :, 3] = alpha_channel
    # Set all channels to 0 if alpha is 0 for that pixel
    alpha_zero = (alpha_channel == 0)
    image[alpha_zero] = 0

    # Crop away any parts of the image that have 0 alpha
    if crop:
        object_pixels = np.where(~alpha_zero)
        xmin, xmax = object_pixels[0].min(), object_pixels[0].max()
        ymin, ymax = object_pixels[1].min(), object_pixels[1].max()
        image = image[xmin:xmax, ymin:ymax]
        
    return image

In [5]:
def augment_images(images):
    """
    Augment transparent object images using imgaug
    """
    seq = iaa.Sequential(
        [
            # Horizontal flips
            iaa.Fliplr(0.5),
            # Small gaussian blur 
            iaa.Sometimes(
                0.5,
                iaa.GaussianBlur(sigma=(0, 0.5))
            ),
            # Strengthen or weaken the contrast in each image.
            iaa.LinearContrast((0.75, 1.5)),
            # Add gaussian noise
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
            # Scale/zoom and rotate
            iaa.Affine(
                scale={"x": (0.6, 1), "y": (0.6, 1)},
                rotate=(-10, 10)
            )
        ],
        random_order=True
    )
    
    # Pad images, augment and crop to bounding box using alpha channel
    images = [pad_image(img, padding=50) for img in images]
    images_aug = seq(images=images)
    images_aug = [restore_alpha(img, crop=True) for img in images_aug]
    
    return images_aug

In [6]:
def load_textures(texture_dir):
    # Texture image file paths
    texture_folders = os.listdir(texture_dir)
    texture_folders.remove('.DS_Store')

    # Recursively get texture image file paths
    texture_files = []
    for folder in texture_folders:
        folder_files = os.listdir(os.path.join(texture_dir, folder))
        for file in folder_files:
            if file != '.directory':
                texture_files.append(os.path.join(texture_dir, folder, file))

    # Load texture images
    textures = []
    for f in texture_files:    
        img = cv2.imread(f, cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
        textures.append(img)

    return textures

In [8]:
def add_background(obj, bg, x, y):
    """
    Add an object to a background at position x, y
    The background will be resized to be square and a multiple of 32
    x, y and the object size will be adjusted to fit the object on the background
    
    Arguments:
    - obj - numpy array - object image to overlay on the background
    - bg - numpy array - background image
    - x - int - x coordinate for the top left of the image
    - y - int - y coordinate for the top left of the image
    
    Returns:
    - img - numpy array - combined images
    - bbox - dict - bounding box for the object
    """
    obj = obj.copy()
    bg = bg.copy()
    
    # Resize background to be a square with length equal to a multiple of 32
    bg_height, bg_width = bg.shape[0], bg.shape[1]
    bg_length = int(min(bg_height, bg_width) / 32)
    bg = bg[:bg_length, :bg_length]
    
    # Size the object so that it fits on the background
    obj_height, obj_width = obj.shape[0], obj.shape[1]
    while (obj_height >= bg_length * 0.9) or (obj_width >= bg_length * 0.9):
        obj = obj[::2, ::2]
        obj_height, obj_width = obj.shape[0], obj.shape[1]
    
    # Merge the object with the background (ensure that it fits)
    x = x % (bg_length - obj_width)
    y = y % (bg_length - obj_height)
    obj = Image.fromarray(obj)
    img = Image.fromarray(bg)
    img.paste(obj, (x, y), obj)
    # Bounding box for training
    bbox = {'xmin': x, 'xmax': x + obj_width, 'ymin': y, 'ymax': y + obj_height}
    
    return img, bbox

In [82]:
def generate_images(objects, backgrounds, sample_size=1, output_dir='../data/processed/'):
    """
    Generate images from object and background images 
    
    Arguments:
    - objects: list of object images with transparent backgrounds
    - backgrounds: list of background images
    - sample_size: number of images to generate
    """
    # Output directories
    time = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
    output_dir = output_dir + time
    image_dir = os.path.join(output_dir, 'images')
    bbox_dir = os.path.join(output_dir, 'bboxes')
    for folder in [output_dir, image_dir, bbox_dir]:
        os.mkdir(folder)
    print(f'Saving images and bounding boxes at: {output_dir}')
    
    # Generate training images
    for i in range(sample_size):
        # Random sample
        obj = np.random.choice(objects)
        bg = np.random.choice(backgrounds)
        x = np.random.randint(0, bg.shape[1])
        y = np.random.randint(0, bg.shape[0])
        # Generate image
        training_image, bbox = add_background(obj, bg, x, y)

        # Save image and bounding box
        training_image.save(os.path.join(image_dir, f'image-{i}.png'))
        json.dump(bbox, open(os.path.join(bbox_dir, f'bbox-{i}.json'), 'w'))

In [ ]:
# Load images
print('Loading object images...')
images = load_images('../data/interim/images/')
# Preview
for image in images[:2]:
    preview(image, size=50)

# Augment images (twice, then append the original images)
print('Augmenting object images...')
images_aug = augment_images(images)
images_aug.extend(augment_images(images))
images_aug.extend(images)
# Preview
for image in images_aug[:2]:
    preview(image, size=50)
    
# Load textures
print('Loading texture images...')
textures = load_textures('../data/raw/dtd/images/')
# Preview
for texture in textures[:2]:
    preview(texture, size=50)

In [84]:
# Generate images
print('Generating images...')
generate_images(images_aug, textures, sample_size=10000)
print('Done')

Generating images...
Saving images and bounding boxes in: ../data/processed/200427_135532


KeyboardInterrupt: 